In [ ]:
#@title <font color='#018ada'>[1] Clonar repositorio e instalar dependencias</font> { display-mode: "form" }
%cd /content
# Clonar el repositorio
!git clone https://github.com/Omarleel/Demiset
# Accede al proyecto
%cd Demiset
# Instalar dependencias
!pip install -r requirements.txt

In [ ]:
#@title <font color='#018ada'>[2] Parámetros de entrada</font> { display-mode: "form" }
import os
import time
from Applications.AudioProcessing import AudioProcessing
from Applications.NoiseReducer import NoiseReducer
from Applications.RemoteFileDownloader import RemoteFileDownloader
from Applications.SilenceRemover import SilenceRemover
from Applications.VoiceExtractor import VoiceExtractor
from Applications.Zipper import Zipper
from Applications.GoogleDriveManager import GoogleDriveManager
import zipfile

def main():
    input_folder = "/content" #@param {type:"string"}
    input_url = 'https://www.twitch.tv/...' #@param {type:"string"}
    output_folder = "/content/Outputs" #@param {type:"string"}
    noise_threshold = 30 #@param {type:"slider", min:0, max:100, step:1}
    ms_split = 15000 #@param {type:"integer"}
    enhance_audio = False #@param {type:"boolean"}

    remote_file_downloader = RemoteFileDownloader()
    audio_processing = AudioProcessing()
    noise_reducer = NoiseReducer()
    voice_extractor = VoiceExtractor()
    silence_remover = SilenceRemover()
    zipper = Zipper()
    google_drive_manager = GoogleDriveManager()

    start_time = time.time()
    # Descargar archivo remoto
    ruta_archivo_descargado = remote_file_downloader.download(input_url, input_folder)
    if ruta_archivo_descargado.endswith('.zip'):
        # Obtener la ruta del directorio donde se extraerá el archivo ZIP
        output_directory = os.path.dirname(ruta_archivo_descargado)
        # Extraer el archivo ZIP en el mismo directorio
        with zipfile.ZipFile(ruta_archivo_descargado, 'r') as zip_ref:
            zip_ref.extractall(output_directory)
    print(f"Ruta del archivo descargado: {ruta_archivo_descargado} (Tiempo: {time.time() - start_time} segundos)")

    start_time = time.time()
    # Combinar todos los audios en un solo archivo
    ruta_audio_combinado = audio_processing.combine_audio(input_folder, output_folder)
    print(f"Ruta del audio combinado: {ruta_audio_combinado} (Tiempo: {time.time() - start_time} segundos)")

    start_time = time.time()
    # Extraer la voz del audio
    ruta_audio_voz = voice_extractor.extract_vocals(ruta_audio_combinado, output_folder)
    print(f"Ruta del audio de voz extraído: {ruta_audio_voz} (Tiempo: {time.time() - start_time} segundos)")

    start_time = time.time()
    if noise_threshold > 0:
        # Reducir ruido del audio
        ruta_audio_sin_ruido = noise_reducer.reduce_noise(ruta_audio_voz, output_folder, noise_threshold)
        print(f"Ruta del audio sin ruido: {ruta_audio_sin_ruido} (Tiempo: {time.time() - start_time} segundos)")
    else:
        ruta_audio_sin_ruido = ruta_audio_voz

    start_time = time.time()
    # Eliminar silencios del audio procesado
    ruta_sin_silencio = silence_remover.remove_silence(ruta_audio_sin_ruido, output_folder)
    print(f"Ruta del audio sin silencios: {ruta_sin_silencio} (Tiempo: {time.time() - start_time} segundos)")

    start_time = time.time()
    # Mejorar la calidad del audio
    if enhance_audio:
        # Mejorar la calidad del audio
        ruta_audio_mejorado = audio_processing.enhance_audio(ruta_sin_silencio, output_folder)
        print(f"Ruta del audio mejorado: {ruta_audio_mejorado} (Tiempo: {time.time() - start_time} segundos)")
    else:
        ruta_audio_mejorado = ruta_sin_silencio

    start_time = time.time()
    # Dividir archivo en audios de 15 segundos
    output_folder_dataset = os.path.join(output_folder, "dataset")
    ruta_audio_dividido = audio_processing.split_audio(ruta_audio_mejorado, output_folder_dataset, ms_split)
    print(f"Ruta del audio dividido: {ruta_audio_dividido} (Tiempo: {time.time() - start_time} segundos)")

    start_time = time.time()
    # Comprimir dataset en un zip
    ruta_dataset_comprimido = zipper.zip_files(ruta_audio_dividido, "dataset.zip")
    output_remote_folder = os.path.join("Demiset", output_folder.lstrip("/"))
    google_drive_manager.upload_file(ruta_dataset_comprimido, output_remote_folder)
    print(f"Dataset creado en: {ruta_dataset_comprimido} (Tiempo: {time.time() - start_time} segundos)")

if __name__ == "__main__":
    main()